In [1]:
# Path to code repository
code_repo_dir = '/Users/user/Documents/Codes/H5N1/H5N1-State-and-County-Simulations-and-Analysis/'

In [2]:
# Loading necessary packages
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.ticker import FuncFormatter
from mpl_toolkits.axes_grid1 import make_axes_locatable

/Users/user/miniconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


# 1. Computing number of dairy cows in each dairy premise

In [3]:
# File paths. Change according to your local directory paths
input_sub_dir_1 = 'data/raw/'    
input_sub_dir_2 = 'data/processed/'

output_sub_dir_1 = 'data/processed/'
output_sub_dir_2 = 'outputs/figures/'

input_filename_1 = 'cattle_FLAPS_for_USAMMv3.txt'
input_filepath_1 = code_repo_dir + input_sub_dir_1 + input_filename_1


output_filename_1 = 'dairy_cows_by_premises.csv'
output_filepath_1 = code_repo_dir + output_sub_dir_1 + output_filename_1

county_shapefile_path = code_repo_dir + input_sub_dir_1 + 'gadm41_USA_shp/gadm41_USA_2.shp'
state_shapefile_path = code_repo_dir + input_sub_dir_1 + 'gadm41_USA_shp/gadm41_USA_1.shp'

In [4]:
# Load the FLAPS file that contain information of cattle per premises (eg. farm)
cattle_by_premises_df = pd.read_csv(input_filepath_1, sep = "\t")
cattle_by_premises_df[cattle_by_premises_df.type == "d"]

,Id,County,X,Y,Lat,Lon,type,b,d
53,757624,1001,844489,-8813162,32.590559,-86.803342,d,0,16
118,757844,1001,875641,-8810000,32.591639,-86.467205,d,0,13
126,757842,1001,875678,-8815462,32.543002,-86.472609,d,0,15
229,756096,1003,765523,-9041656,30.615888,-87.854986,d,0,4
245,756766,1003,799779,-9063597,30.393238,-87.517768,d,0,2
...,...,...,...,...,...,...,...,...,...
815615,729298,56043,-972175,-7522047,43.910056,-108.098869,d,0,2
815616,729318,56043,-964822,-7520318,43.933834,-108.010241,d,0,2
815749,730103,56045,-656624,-7562069,43.857095,-104.114942,d,0,4
815781,730094,56045,-655476,-7534925,44.100281,-104.130074,d,0,7


In [5]:
# Filter rows by cattle type 'd' (for dairy cow) and select the columns: Premises Id (Id), 
# County FIPS (County), Lat, Lon, Number of dairy cattle in premises (d) 
dairy_cows_by_premises_df = cattle_by_premises_df.loc[
    cattle_by_premises_df.type == "d",  ['Id', 'County', 'Lat', 'Lon', 'd']
].reset_index(drop=True)
dairy_cows_by_premises_df.to_csv(output_filepath_1, index=False)

# 2. Computing number of dairy cows by counties

In [ ]:
input_filename_2 = 'dairy_cows_by_premises.csv'
input_filepath_2 = code_repo_dir + input_sub_dir_2 + input_filename_2

output_filename_2 = 'dairy_cows_by_counties.csv'
output_filepath_2 = code_repo_dir + output_sub_dir_1 + output_filename_2

print(input_filepath_2, output_filepath_2)

In [ ]:
dairy_cows_by_premises_df = pd.read_csv(input_filepath_2)
dairy_cows_by_premises_df

In [ ]:
# Load US shape file with counties as administrative regions and trim to mainland 
counties_shp_gdf = gpd.read_file(county_shapefile_path)
counties_shp_gdf = counties_shp_gdf.cx[-130:-60, 25:50]
counties_shp_gdf

In [ ]:
# Convert the dairy_cows_by_premises dataframe to a geodataframe
geometry = [Point(xy) for xy in zip(dairy_cows_by_premises_df['Lon'], dairy_cows_by_premises_df['Lat'])]
dairy_cows_by_premises_gdf = gpd.GeoDataFrame(dairy_cows_by_premises_df, geometry=geometry)
dairy_cows_by_premises_gdf

In [ ]:
# Set the CRS of the cattle data to match the states shapefile if they are different
# Assuming states_gdf is in EPSG:4326, update if necessary
dairy_cows_by_premises_gdf.set_crs(epsg=4326, inplace=True)
dairy_cows_by_premises_gdf.to_crs(counties_shp_gdf.crs, inplace=True)

In [ ]:
# Perform a spatial join to aggregate cows data by counties
# Spatial join to get county names
dairy_cows_by_premises_with_county_gdf = gpd.sjoin(dairy_cows_by_premises_gdf, counties_shp_gdf, how='left', op='within')
dairy_cows_by_premises_with_county_gdf

In [ ]:
# Aggregate the number of dairy cows by county
dairy_cows_by_county = dairy_cows_by_premises_with_county_gdf.groupby(['NAME_1','NAME_2'])['d'].sum().reset_index()
dairy_cows_by_county.sort_values(by = 'd', ascending = False).head(50)

In [ ]:
dairy_cows_by_county.to_csv(output_filepath_2, index=False)

# 3. Plotting county-wise dairy cows distribution

In [ ]:
# Merge the aggregated data with the counties GeoDataFrame
counties_dc_geom_gdf = counties_shp_gdf.merge(dairy_cows_by_county, 
                                        left_on=['NAME_1', 'NAME_2'], right_on=['NAME_1','NAME_2'], how='left')
counties_dc_geom_gdf.head(5)

In [ ]:
fig_filename = 'dairy_cows_by_county_logscale.png'
fig_path = code_repo_dir + output_sub_dir_2 + fig_filename
print(fig_path)

In [ ]:
counties_dc_geom_gdf['log_d'] = np.log(counties_dc_geom_gdf['d'])
counties_dc_geom_gdf

In [ ]:
# Plot the states with colors corresponding to the number of dairy cows
fig, ax = plt.subplots(figsize=(12, 10))
counties_dc_geom_gdf.boundary.plot(ax=ax, linewidth=1, color='black', alpha = 0.2)
state_borders = gpd.read_file(state_shapefile_path)
state_borders = state_borders.cx[-130:-60, 25:50]

# Plot state borders
state_borders.plot(ax=plt.gca(), color='none', edgecolor='black', linewidth=1.5, alpha =1)

# Plot the aggregated dairy cows count per county
counties_dc_geom_gdf.plot(column='log_d', ax=ax, legend=True, 
                legend_kwds={'shrink': 0.4, 'label': "Dairy Cows per County (log-scale)",
                             'orientation': "horizontal", 
                             'pad': 0.1, 'fmt': '%.0f'},
                cmap = "OrRd"
        )

cbar = ax.get_figure().axes[-1] 
cbar.set_xlabel("Dairy Cows per County (log-scale)", fontsize=12)  # Adjust fontsize here

plt.title('Population of Dairy Cows by County in the US (log scale)', fontsize = 18)
plt.xlabel('Longitude', fontsize = 14)
plt.ylabel('Latitude', fontsize = 14)
plt.savefig(fig_path, dpi = 600)
plt.show()

# 4. Computing number of dairy cows by state

In [ ]:
output_filename_3 = 'dairy_cows_by_state.csv'
output_filepath_3 = code_repo_dir + output_sub_dir_1 + output_filename_3
output_filepath_3

In [ ]:
dairy_cows_by_state = counties_dc_geom_gdf.groupby('NAME_1')['d'].sum().reset_index()
dairy_cows_by_state["d"] = dairy_cows_by_state["d"].astype(int)
dairy_cows_by_state.sort_values(by = 'd', ascending = False)

In [ ]:
dairy_cows_by_state.to_csv(output_filepath_3, index=False)